In [1]:
import numpy as np
import pandas as pd
import time

## PAMR

Pamr: Passive aggressive mean reversion strategy for portfolio selection, 2012.

Reference:
B. Li, P. Zhao, S. C.H. Hoi, and V. Gopalkrishnan.
http://www.cais.ntu.edu.sg/~chhoi/paper_pdf/PAMR_ML_final.pdf

#### Algorithm : PAMR
INPUT: 𝜖: sensitivity parameter; C: aggressiveness parameter

PROCEDURE
1. Initialize $𝒃_𝟏 =(\frac{1}{𝑚},⋯,\frac{1}{𝑚})$ 
2. for t = 1, 2,...,n do
3. $\quad$Receive stock price relatives: $𝑥_𝑡=(𝑥_{t1},⋯,𝑥_{tm})$
4. $\quad$Suffer loss: $ℓ_𝜖^𝑡=max⁡( 0, 𝒃_𝒕 · 𝒙_𝒕 − 𝜖)$
5. $\quad$Set parameters: $$
\tau_t=\begin{cases}
\frac{ℓ_𝜖^𝑡}{‖x−\bar x_𝑡 1‖^2},& (PAMR) \\ min(C, \frac{ℓ_𝜖^𝑡}{‖x−\bar x_𝑡 1‖^2}),& (PAMR-1) \cr \frac{ℓ_𝜖^𝑡}{‖x−\bar x_𝑡 1‖^2 + \frac{1}{2C}},& (PAMR-2)
\end{cases}
$$
6. $\quad$Update portfolio: $$𝒃_{𝑡+1}= 𝒃_𝑡 - \tau_t (x−\bar x_𝑡 1)$$
7. $\quad$Normalize portfolio: $$𝒃_{𝑡+1}=argmin‖𝒃−𝒃_𝑡 ‖^2$$
8. end for

END

In [38]:
data=pd.read_csv('./Data_OLPS/nyse-o.csv',encoding="gb18030")
print('data.shape：',data.shape) 

# nyse-o , nyse-n, djia, tse, sp500

data.shape： (5651, 36)


In [39]:
N=data.shape[0]
d=data.shape[1]
x=np.zeros((N,d))
x=data.to_numpy()
x

array([[1.01515, 1.02765, 1.04183, ..., 1.00578, 0.99697, 0.99752],
       [1.01493, 1.04036, 0.98905, ..., 1.00958, 0.99088, 1.00248],
       [1.     , 0.97629, 0.97786, ..., 1.     , 1.02761, 0.99752],
       ...,
       [0.99029, 0.9966 , 0.99605, ..., 0.99216, 1.00461, 0.99273],
       [0.99265, 1.00683, 1.     , ..., 0.99209, 1.02752, 1.00366],
       [0.99753, 1.00339, 1.01984, ..., 1.01195, 1.     , 0.99635]])

In [40]:
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m

    return np.maximum(y-tmax,0.)

In [45]:
start = time.time()
cum_ret=1
C=500
gamma = 0.0001
b=np.ones(d)/d
daily_r=np.ones(N)
epsilon = 0.5 
tc=0.0007
for t in range(N-1):
    le = np.maximum(0,np.dot(x[t],b) - epsilon)
    denominator = np.sum((x[t] - np.mean(x[t]))**2)
    tau = le / denominator                         # PAMR
#     tau = np.minimum(C, le / denominator)          # PAMR-1
#     tau = le / (denominator + 0.5 / C)             # PAMR-2
    b = b-tau * (x[t] - np.mean(x[t])) 
    b = simplex_proj(b)
#     tc = gamma/2 * (np.abs(((x[t+1]*b)/ (np.dot(x[t+1],b)))[1:] - b[1:])).sum()
    daily_r[t] = np.dot(x[t+1],b) #* (1-tc)
    cum_ret = cum_ret * daily_r[t] 
print("Cumulative return:",cum_ret,"-----> Expressed by scientific counting:",f'{cum_ret:1.2e}') 

end = time.time()
print("Program running time:",end-start)

Cumulative return: 5062991996190654.0 -----> Expressed by scientific counting: 5.06e+15
Program running time: 0.54996657371521


In [46]:
daily_r = pd.DataFrame(daily_r,index = data.index,columns=['daily return'])
# print(daily_r)
mean_daily = daily_r['daily return'].mean()
print("Average daily rate of return:",mean_daily)
mean_return_annualized = mean_daily**252 - 1
# print("mean_return_annualized:",mean_return_annualized)

std_daily = np.std(np.array(daily_r))
variance_daily = std_daily ** 2
std_annualized = std_daily*np.sqrt(252)
variance_annualized = std_annualized ** 2
volatility = std_daily * np.sqrt(252)
print("Volatility is: ",volatility)
SR = mean_daily/std_daily
print("Daily Sharpe ratio is: ",SR)
ASR=np.sqrt(252)*SR
maximum = np.maximum.accumulate(np.array(daily_r))
MDD = ((maximum - np.array(daily_r)) / maximum).max()
print("Max Drawdown is: ",MDD)
CS = mean_return_annualized/MDD
print("Calmar Ratio is: ",CS)

Average daily rate of return: 1.0069198319928963
Volatility is:  0.5112122875470325
Daily Sharpe ratio is:  31.267552019320863
Max Drawdown is:  0.381751023232395
Calmar Ratio is:  12.2718134208421
